In [1]:
# Pre-Processing

In [1]:
import pandas as pd
import os

In [2]:
# read in data (TODO: all years)
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat.head(10)

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,1.0,Fernando Abad*\abadfe01,28.0,OAK,AL,7,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
1,2.0,Bobby Abreu*\abreubo01,40.0,NYM,NL,78,155,133,12,33,...,0.338,0.680,97.0,45,3,0,0,2,0,H9/7D
2,3.0,José Abreu\abreujo02,27.0,CHW,AL,145,622,556,80,176,...,0.581,0.964,173.0,323,14,11,0,4,15,*3D/H
3,4.0,Tony Abreu#\abreuto01,29.0,SFG,NL,3,4,4,0,0,...,0.000,0.000,-100.0,0,1,0,0,0,0,/45H
4,5.0,Alfredo Aceves\aceveal01,31.0,NYY,AL,4,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5,6.0,Dustin Ackley*\ackledu01,26.0,SEA,AL,143,542,502,64,123,...,0.398,0.692,99.0,200,10,3,3,2,1,*7H/D
6,7.0,Cristhian Adames#\adamecr01,22.0,COL,NL,7,15,15,1,1,...,0.067,0.133,-65.0,1,1,0,0,0,0,/64H
7,8.0,Lane Adams\adamsla01,24.0,KCR,AL,6,3,3,1,0,...,0.000,0.000,-100.0,0,0,0,0,0,0,/H8D7
8,9.0,Matt Adams*\adamsma01,25.0,STL,NL,142,563,527,55,152,...,0.457,0.779,116.0,241,9,3,0,7,5,*3H
9,10.0,Mike Adams\adamsmi03,35.0,PHI,NL,22,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1


In [7]:
def easy_name(name):
    name = name.split("\\")[0]
    if name[-1] == "#" or name[-1] == "*":
        name = name[:-1]
    return name

def data_etl(dat, year):
    # Remove last row: league averages
    dat = dat.iloc[:-1]
    
    # Remove dupes, add unique ID
    for index, row in dat.iterrows():
        if row["Tm"] == "TOT":
            dat = dat.drop(dat[(dat['Name'] == row["Name"]) & (dat["Tm"] != "TOT")].index) 
        #row["ID"] = row["Name"] + str(year)

    # Exclude pitchers
    dat = dat[dat["Pos Summary"] != "1"]
    dat = dat[dat["Pos Summary"] != "/1"]

    # Exclude players with < 50 plate appearances
    dat = dat[dat["PA"] >= 50]

    # Make names easier to read
    dat['Name'] = dat['Name'].apply(easy_name)

    # Add Year Column, ID Column
    dat["Year"] = year
    dat["ID"] = dat["Name"] + dat["Year"].apply(str)
    
    # Drop columns that we don't intend on using
    cols_to_drop = ['Rk', 'Lg', 'PA', 'AB', '2B', '3B', 'CS', 'BB', 'OBP', 'OPS+', 'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'Pos Summary']
    dat.drop(cols_to_drop, axis=1, inplace=True)
    
    return dat


In [8]:
# ETL and concatenate data from all years
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat = data_etl(dat, 2014)
print(len(dat))
for i in [2015, 2016, 2017, 2018, 2019, 2021]:
    dat_new = pd.read_csv(os.path.join("data", "mlb_data_" + str(i) + ".csv"))
    dat_new = data_etl(dat_new, i)
    dat = pd.concat([dat, dat_new])
    print(len(dat))

522
1044
1567
2092
2616
3165
3708


In [9]:
# Make sure all years are in data
dat[dat["Name"] == "Kolten Wong"]

,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,BA,SLG,OPS,Year,ID
1379,Kolten Wong,23.0,STL,113,52,100,12,42,20,71,0.249,0.388,0.680,2014,Kolten Wong2014
1472,Kolten Wong,24.0,STL,150,71,146,11,61,15,95,0.262,0.386,0.707,2015,Kolten Wong2015
1444,Kolten Wong,25.0,STL,121,39,75,5,23,7,52,0.240,0.355,0.682,2016,Kolten Wong2016
1424,Kolten Wong,26.0,STL,108,55,101,4,42,8,60,0.285,0.412,0.788,2017,Kolten Wong2017
1510,Kolten Wong,27.0,STL,127,41,88,9,38,6,60,0.249,0.388,0.720,2018,Kolten Wong2018
1526,Kolten Wong,28.0,STL,148,61,136,11,59,24,83,0.285,0.423,0.784,2019,Kolten Wong2019
1695,Kolten Wong,30.0,MIL,116,70,121,14,50,12,83,0.272,0.447,0.783,2021,Kolten Wong2021


In [12]:
# Add all-star data
as_dat = pd.read_csv(os.path.join("data", "all_star_data.csv"))
ids = []
for index, row in as_dat.iterrows():
    ids.append(str(row["Name"]) + str(row["Year"]))

allstars = []
for index, row in dat.iterrows():
    if row["ID"] in ids:
        allstars.append(1)
    else:
        allstars.append(0)
dat["allstars"] = allstars
dat.head(10)

,Name,Age,Tm,G,R,H,HR,RBI,SB,SO,BA,SLG,OPS,Year,ID,allstars
1,Bobby Abreu,40.0,NYM,78,12,33,1,14,1,21,0.248,0.338,0.680,2014,Bobby Abreu2014,0
2,José Abreu,27.0,CHW,145,80,176,36,107,3,131,0.317,0.581,0.964,2014,José Abreu2014,1
5,Dustin Ackley,26.0,SEA,143,64,123,14,65,8,90,0.245,0.398,0.692,2014,Dustin Ackley2014,0
8,Matt Adams,25.0,STL,142,55,152,15,68,3,114,0.288,0.457,0.779,2014,Matt Adams2014,0
10,Jim Adduci,29.0,TEX,44,13,17,1,8,3,27,0.168,0.228,0.467,2014,Jim Adduci2014,0
11,Ehire Adrianza,24.0,SFG,53,10,23,0,5,1,22,0.237,0.299,0.578,2014,Ehire Adrianza2014,0
14,Nick Ahmed,24.0,ARI,25,9,14,1,4,0,10,0.200,0.271,0.504,2014,Nick Ahmed2014,0
16,Arismendy Alcantara,22.0,CHC,70,31,57,10,29,8,93,0.205,0.367,0.621,2014,Arismendy Alcantara2014,0
18,Abraham Almonte,25.0,TOT,59,19,47,3,15,4,60,0.230,0.333,0.609,2014,Abraham Almonte2014,0
22,Yonder Alonso,27.0,SDP,84,27,64,7,27,6,36,0.240,0.397,0.682,2014,Yonder Alonso2014,0


In [10]:
# Save master data file
dat.to_csv("master_data.csv")